#### **事前準備**
今回の RAG 開発で用いる変数を事前に準備しておきます。  
※ 以下のような内容の settings.py を事前に準備しておいてください
```python
OPENAI_API_KEY = "*"
OPENAI_MODEL = "gpt-4o"
OPENAI_EMBEDDING_MODEL = "text-embedding-ada-002"
```

In [1]:
from settings import (
    OPENAI_API_KEY,
    OPENAI_MODEL,
    OPENAI_EMBEDDING_MODEL,
)
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

#### **データの加工**
今回 LLM に読み込ませるデータを ./docs フォルダに用意しておきました。  
まずはこの pdf ファイルをテキスト化します。

In [2]:
from PyPDF2 import PdfReader

reader = PdfReader("./docs/Labor.pdf")

page_text = []
print("now loading:", end="")
for page in reader.pages:
    text = page.extract_text().replace("\n", " ").replace(" ", "")
    page_text.append(text)
    print("=", end="")

now loading:================================

#### **トークン数の確認**
LLM は、文章が与えられると、それを一度 token と言われる数値のまとまりに変換します。  
基本的に、料金や入力の可否はこの token の数によって決まるので、これが最大になっているページの token 数を見てみます。

In [3]:
import tiktoken

encoding = tiktoken.encoding_for_model(OPENAI_MODEL)
max_token = 0
for text in page_text[2:]:
    token_integers = encoding.encode(text)
    num_tokens = len(token_integers)
    if num_tokens > max_token:
        max_token = num_tokens
print("max_token:", max_token)

max_token: 1682


今回使う GPT-4o という OpenAI のモデルでは 128,000 token 数まで入力できるので、今回他のは特別な必要なさそうです。

#### **ページごとに文章をベクトル化して jsonl に保存**
今回作成しようとしていた RAG とは
>   プロンプトがユーザーから与えられた際に、その回答を作成するのに必要そうなデータをプロンプトと一緒に LLM に与えることで、回答の精度向上を目指す。

ものでした。  

「回答を作成するのに必要そうなデータ」を見分けるために文章の類似度を用います。  
プロンプトに似た内容が書いてあるページの情報だけを LLM に与えようということです。  
そこで、文章の類似度を評価するために、一度文章をベクトル化します。  
これを embedding といい、今回は OpenAI が提供しているモデルを使って行います。

※ 多少お金がかかるので2度は実行しない

```python
import json
from openai import OpenAI

OPENAI_CLIENT = OpenAI()

output_file = "./json/data.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for text in page_text[3:]:
        json_data = {
            "text": text,
            "embedding": OPENAI_CLIENT.embeddings.create(
                input=text,
                model=OPENAI_EMBEDDING_MODEL
            ).data[0].embedding
        }
        json_line = json.dumps(json_data, ensure_ascii=False)
        f.write(json_line + "\n")
```